# Compiling and Submitting Circuits onto Hilbert via Cirq


<a target="_blank" href="https://colab.research.google.com/github/SupertechLabs/client-superstaq/blob/main/docs/source/optimizations/cq/cq_compile_css.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>
<a target="_blank" href="https://mybinder.org/v2/gh/SupertechLabs/client-superstaq/HEAD?labpath=docs/source/optimizations/cq/cq_compile_css.ipynb">
  <img src="https://mybinder.org/badge_logo.svg" alt="Launch Binder" align="left" style="margin-left: 10px;"/>
</a>

This tutorial will showcase how to compile and submit a circuit onto Infleqtion's hardware, Hilbert, using the cirq-superstaq client. First, you must import the necessary packages to get started. 

## Import Requirements

In [15]:
import os
import json

try:
    import cirq
    import cirq_superstaq as css
    
except ImportError:
    print("installing cirq-superstaq...")
    !pip install --quiet cirq-superstaq
    import cirq
    import cirq_superstaq as css
    
    print("installed cirq-superstaq")

Next, we must instantiate our provider in cirq-superstaq, using a SuperstaQ token. You can get your SuperstaQ token from https://superstaq.super.tech. 

In [45]:
token = "Insert superstaq token that you received from https://superstaq.super.tech"
service = css.Service(token)

## Create a Circuit

In [46]:
qubits = cirq.LineQubit.range(2)
circuit1 = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(qubits[0]))
print(circuit1)

0: ───H───@───M───
          │
1: ───────X───────


## Single Circuit Compilation

In [47]:
compiler_output = service.cq_compile(circuit1)

/home/salahedeen/PycharmProjects/my_project/new_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


SuperstaQException: Status code: 404, Message: 'Non-retriable error making request to SuperstaQ API, The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again. You have requested this URI [/v0.2.0/compile] but did you mean /v0.2.0/cq_compile or /v0.2.0/aqt_compile or /v0.2.0/ibmq_compile ?'

In [48]:
print(compiler_output.circuit)

(0, 0): ───RGate(0.5π, -0.5π)───@───RGate(-0.25π, -0.5π)───────────RGate(0.25π, -0.5π)───M('q(0)')───────────
           │                    │   │                              │
(0, 1): ───#2───────────────────@───#2─────────────────────Rz(π)───#2────────────────────────────────Rz(π)───


## Enter Hilbert credentials

To get your ColdQuanta credentials onto your local machine, run this:

In [53]:
# uncomment command below to get auth token, ! will run cmd line
# !python -m coldquanta.qc_common_api.cq_authorize

To make sure your credentials have been saved to your machine, run the code below:

In [52]:
!cat ~/.coldquanta/cq_auth.json

Now load in your Hilbert credentials:

In [50]:
with open(os.path.expanduser("~/.coldquanta/cq_auth.json"), "r") as file:
    data = json.load(file)
options = {"cq_token": data}

## Single Circuit Submission

The code below will submitt he circuit to the noiseless Hilbert simulator. If you would like to run on Hilbert, change the "target" argument in job_css from "cq_hilbert_simulator" to "cq_hilbert_qpu". It is recommended to first submit to the simulator for testing to ensure your code runs before submitting to Hilbert. 

In [51]:
job_css = service.create_job(
    circuit=circuit1, repetitions=100, target="cq_hilbert_simulator", options=options
)
result_css = job_css.counts()
print(f"Counts from cirq-superstaq submission: {result_css}")

/home/salahedeen/PycharmProjects/my_project/new_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


SuperstaQException: Status code: 400, Message: 'Non-retriable error making request to SuperstaQ API, Your CQ token is invalid. If you are using SuperstaQ without the clients, make sure that the project and org id you are setting are correct. If you're accessing SuperstaQ via the open source clients, your account is not associated with the HilbertEndToEnd project. Contact info@super.tech to be added to this project.'

## Multiple circuit compilation

In [7]:
compiler_output = service.cq_compile([circuit, circuit1])

In [8]:
print(compiler_output.circuits[0])
print()
print(compiler_output.circuits[1])

(0, 0): ───RGate(0.5π, -0.5π)───M───

(0, 0): ───RGate(0.5π, -0.5π)───@───M───RGate(-0.25π, -0.5π)───────────RGate(0.25π, -0.5π)───────────
           │                    │       │                              │
(0, 1): ───#2───────────────────@───────#2─────────────────────Rz(π)───#2────────────────────Rz(π)───
